# Simple decorator

In [1]:
# define decorator

In [5]:
def uppercase(func):
    def wrapper():
        orignal_result = func()
        modified_result = orignal_result.upper()
        return modified_result
    return wrapper

In [8]:
# define main function 

In [9]:
@uppercase
def greet():
    return "Hello!"

In [10]:
# calling function

In [7]:
greet()

'HELLO!'

#  Multiple decorator

In [1]:
def strong(func):
    def wrapper():
        return "<strong>" + func() + "</strong>"
    return wrapper

def emphasis(func):
    def wrapper():
        return "<em>" + func() + "</em>"
    return wrapper

In [2]:
@strong
@emphasis
def greet():
    return "Hello!"

In [3]:
greet()

'<strong><em>Hello!</em></strong>'

this is like doing

In [11]:
def clean_greet():
    return "Hello!"

In [12]:
some_function = strong(emphasis(clean_greet))

In [13]:
some_function()

'<strong><em>Hello!</em></strong>'

# Arguments

In [14]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

In [19]:
def trace(func):
    def wrapper(*args, **kwargs):
        print("TRACE: calling {}() with {}, {}".format(func.__name__, \
                                                      args, kwargs))
        
        original_result = func(*args, **kwargs)
        
        print("TRACE: {}() returned {}".format(func.__name__, original_result))\
        
        return original_result
    return wrapper

In [20]:
@trace
def say(name, line):
    return "{}: {}".format(name, line)

In [21]:
say('Jean','hello, world')

TRACE: calling say() with ('Jean', 'hello, world'), {}
TRACE: say() returned Jean: hello, world


'Jean: hello, world'

In [22]:
@trace
def say_it_again(name='', line=''):
    return "{}: {}".format(name, line)

In [23]:
say_it_again(name='Jean', line='Hello today!')

TRACE: calling say_it_again() with (), {'line': 'Hello today!', 'name': 'Jean'}
TRACE: say_it_again() returned Jean: Hello today!


'Jean: Hello today!'

# Data in decorators 

In [17]:
def running_average(func):
    data = {"total": 0, "count": 0}
    def wrapper(*args, **kwargs):
        val = func(*args, **kwargs)
        data["total"] += val
        data["count"] += 1
        print("Average of {} so far: {:.01f}".format(func.__name__, data["total"]/data["count"]))
        return func(*args, **kwargs)
    return wrapper

In [18]:
@running_average
def foo(x):
    return x+2

In [19]:
foo(1)

Average of foo so far: 3.0


3

In [20]:
foo(3)

Average of foo so far: 4.0


5

In [21]:
foo(10)

Average of foo so far: 6.7


12

## getting a peek of data ? 

In [22]:
def running_average(func):
    data = {"total": 0, "count": 0}
    def wrapper(*args, **kwargs):
        val = func(*args, **kwargs)
        data["total"] += val
        data["count"] += 1
        print("Average of {} so far: {:.01f}".format(func.__name__, data["total"]/data["count"]))
        return func(*args, **kwargs)
    wrapper.data = data
    return wrapper

In [23]:
@running_average
def foo(x):
    return x+2

In [24]:
foo(10)

Average of foo so far: 12.0


12

In [25]:
foo.data

{'total': 12, 'count': 1}

because function behave like objects, we can get ride of the **print** statement

In [35]:
def collect_stats(func):
    data = {"total": 0, "count": 0}
    def wrapper(*args, **kwargs):
        val = func(*args, **kwargs)
        data["total"] += val
        data["count"] += 1
        return func(*args, **kwargs)
    wrapper.data = data
    return wrapper

In [36]:
@collect_stats
def new_foo(x):
    return x+2

In [37]:
new_foo(1)
new_foo(10)

12

In [38]:
new_foo.data

{'total': 15, 'count': 2}

# Debugging

metadata, docstring of the orginal function are hidden by the wrapper function

unless !

In [25]:
import functools

def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

In [26]:
@uppercase
def greet():
    """Return a friendly greeting."""
    return "Hello!"

In [27]:
greet()

'HELLO!'

This way we still can reach the documentation of the initial function

In [29]:
greet.__doc__

'Return a friendly greeting.'

In [30]:
greet.__name__

'greet'

# Examples

source: **Powerful Python** by Aaron Maxwell

Let's write a decorator that count how many times a function has been called

In [39]:
def how_many_times(func):
    count = 0
    def wrapper(*args, **kwargs):
        count += 1
        return func(*args, **kwargs)
    return wrapper

In [40]:
@how_many_times
def my_function():
    return "yo"

In [41]:
my_function()

UnboundLocalError: local variable 'count' referenced before assignment

Solution? -> Ok, what if count is a dictionary instead?

In [42]:
def how_many_times_1(func):
    result = {"count": 0}
    def wrapper(*args, **kwargs):
        result["count"] += 1
        return func(*args, **kwargs)
    wrapper.result = result
    return wrapper

In [43]:
@how_many_times_1
def my_function():
    return "yo"

In [46]:
my_function()

'yo'

In [47]:
my_function.result

{'count': 3}

So it does work when it's a dictionary, but it doesn't when it's a simple variable.

Here is the solution to make it work with a simple variable in python 3, using **nonlocal** keyword

In [58]:
def how_many_times_2(func):
    count = 0
    def wrapper(*args, **kwargs):
        nonlocal count
        count += 1
        print("function has been called {} times".format(count))
        return func(*args, **kwargs)
    return wrapper

In [59]:
@how_many_times_2
def my_function():
    return "yo"

In [60]:
my_function()

function has been called 1 times


'yo'

In [61]:
my_function()

function has been called 2 times


'yo'

In [62]:
my_function()

function has been called 3 times


'yo'

# Decorators that take arguments

In [63]:
def add(increment):
    def decorator(func):
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs) + increment
        return wrapper
    return decorator


In [67]:
@add(2)
def power_2(x):
    return x ** 2

In [69]:
power_2(5)

27

In [72]:
@add(4)
def minus_2(x):
    return x - 2

In [73]:
minus_2(5)

7

# Class-based decorators

The secret is to implement a function **\_\_call__**, to make the class behaves like a function.

In [74]:
class Prefixer:
    
    def __init__(self, prefix):
        self.prefix = prefix
    def __call__(self, message):
        return self.prefix + message

In [75]:
instance = Prefixer("Simon says: ")


In [76]:
instance("time to leave!")

'Simon says: time to leave!'

Let's try to use a class-based decorator on the *printlog* function

In [77]:
def printlog(func):
    def wrapper(*args, **kwargs):
        print("calling {}".format(func.__name__))
        return func(*args, **kwargs)
    return wrapper

In [78]:
@printlog
def foo(x):
    return x+2

In [79]:
foo(10)

calling foo


12

In [95]:
class PrintLog:
    
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        print("calling {}".format(self.func.__name__))
        return self.func(*args, **kwargs)


In [96]:
@PrintLog
def baba(x):
    return x+10

In [97]:
baba(20)

calling baba


30

## with arguments

In this case the signature of the class-decorator is a litte bit different as the function is really passed in in the **\_\_call__** method

In [98]:
class add:
    
    def __init__(self, increment):
        self.increment = increment
        
    def __call__(self, function):
        def wrapper(*args, **kwargs):
            return function(*args, **kwargs) + self.increment
        return wrapper

In [101]:
@add(2)
def foo(x):
    return x**2

In [102]:
foo(3)

11

# Decorators for classes

ex: use of \_\_repr__

In [104]:
class Penny:
    value = 1

In [105]:
penny = Penny()
penny

Implementing \_\_repr__

In [115]:
class Penny:
    value = 1
    def __repr__(self):
        return "{}()".format(__class__.__name__)

In [116]:
penny = Penny()
penny

Penny()

Now let's use a class as decorator to automatically add this \_\_repr__ all the time

In [117]:
def autorpr(klass):
    def klass_repr(self):
        return '{}()'.format(klass.__name__)
    klass.__repr__ = klass_repr
    return klass

In [118]:
@autorpr
class Penny:
    value = 1

In [119]:
penny = Penny()
penny

Penny()

# what about class with parameters 

In [ ]:
def autorpr(klass):
    def klass_repr(self):
        return '{}()'.format(klass.__name__)
    klass.__repr__ = klass_repr
    return klass

In [121]:
@autorpr
class NewPenny:
    
    def __init__(self, value):
        self.value = value

In [122]:
new_penny = NewPenny(10)

In [123]:
new_penny

NewPenny()

In [124]:
new_penny.value

10

# Problems of decorators 

One of the problem is that when using a decorator, you loose some information about the original class, such as *name*, *doc* ...

To solve this, we can use the **functools** module and its **wrap** function

### Example showing issue here

In [131]:
def printlog(func):
    def wrapper(*args, **kwargs):
        print("Calling: {}".format(func.__name__))
        return func(*args, **kwargs)
    return wrapper
    
@printlog
def foo(x):
    """we are adding 2 to the input variable"""
    return x*2
    

In [132]:
foo(10)

Calling: foo


20

In [143]:
print("foo.__doc__: {}".format(foo.__doc__))  # where is my doc ?????
print("foo.__name__: {}".format(foo.__name__))  # wrong
print("foo.__module__: {}".format(foo.__module__))
print("foo.__annotations__: {}".format(foo.__annotations__))

foo.__doc__: None
foo.__name__: wrapper
foo.__module__: __main__
foo.__annotations__: {}


###  Solution

In [144]:
import functools
def printlog(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("calling: {}".format(func.__name__))
        return func(*args, **kwargs)
    return wrapper

@printlog
def foo(x):
    """we are adding 2 to the input variable"""
    return x*2


In [145]:
print("foo.__doc__: {}".format(foo.__doc__))  # where is my doc ?????
print("foo.__name__: {}".format(foo.__name__))  # wrong
print("foo.__module__: {}".format(foo.__module__))
print("foo.__annotations__: {}".format(foo.__annotations__))

foo.__doc__: we are adding 2 to the input variable
foo.__name__: foo
foo.__module__: __main__
foo.__annotations__: {}


for class-based decorators, we need to use **update_wrapper**

In [148]:
import functools
class PrintLog:
    def __init__(self, func):
        self.func = func
        functools.update_wrapper(self, func)
    def __call__(self, *args, **kwargs):
        print("Calling with PrintLog decorator: {}".format(self.func.__name__))
        return self.func
    
@PrintLog
def foo(x):
    """we are adding 2 to the input variable"""
    return x*2

In [149]:
foo(10)

Calling with PrintLog decorator: foo


<function __main__.foo(x)>

In [150]:
print("foo.__doc__: {}".format(foo.__doc__))  # where is my doc ?????
print("foo.__name__: {}".format(foo.__name__))  # wrong
print("foo.__module__: {}".format(foo.__module__))
print("foo.__annotations__: {}".format(foo.__annotations__))

foo.__doc__: we are adding 2 to the input variable
foo.__name__: foo
foo.__module__: __main__
foo.__annotations__: {}
